<a href="https://colab.research.google.com/github/makhlufiaero338/tugas-robot/blob/main/w11/Simulasi_informasi_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

In [12]:
from google.colab import files

uploaded = files.upload()

Saving nancy1.jpg to nancy1.jpg


In [2]:
# Helper functions to display images
def display_image(title, img):
    plt.figure(figsize=(8, 6))
    if len(img.shape) == 3:  # Color image
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img, cmap='gray')
    plt.title(title)
    plt.axis('off')
    plt.show()

In [14]:
#################################
# 1. Ekstraksi Garis dengan Hough Transform
#################################
def extract_lines_hough(image_path):
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150, apertureSize=3)

    lines = cv2.HoughLines(edges, 1, np.pi/180, 200)
    if lines is not None:
        for rho, theta in lines[:, 0]:
            a, b = np.cos(theta), np.sin(theta)
            x0, y0 = a * rho, b * rho
            x1 = int(x0 + 1000 * (-b))
            y1 = int(y0 + 1000 * (a))
            x2 = int(x0 - 1000 * (-b))
            y2 = int(y0 - 1000 * (a))
            cv2.line(img, (x1, y1), (x2, y2), (0, 0, 255), 2)

    display_image("Hough Line Transform", img)

In [4]:
#################################
# 2. Template Matching untuk Deteksi Objek
#################################
def template_matching(image_path, template_path):
    img = cv2.imread(image_path)
    template = cv2.imread(template_path, 0)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    res = cv2.matchTemplate(gray, template, cv2.TM_CCOEFF_NORMED)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)

    top_left = max_loc
    h, w = template.shape
    bottom_right = (top_left[0] + w, top_left[1] + h)

    cv2.rectangle(img, top_left, bottom_right, (0, 255, 0), 2)
    display_image("Template Matching", img)

In [5]:
#################################
# 3. Pembuatan Pyramid Gambar
#################################
def create_image_pyramid(image_path):
    img = cv2.imread(image_path)
    pyramid = [img]

    for i in range(3):
        img = cv2.pyrDown(img)
        pyramid.append(img)

    for i, layer in enumerate(pyramid):
        display_image(f"Pyramid Level {i}", layer)

In [6]:
#################################
# 4. Deteksi Lingkaran Menggunakan Hough Transform
#################################
def detect_circles_hough(image_path):
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.medianBlur(gray, 5)

    circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, dp=1.2, minDist=30, param1=50, param2=30, minRadius=10, maxRadius=50)
    if circles is not None:
        circles = np.uint16(np.around(circles))
        for i in circles[0, :]:
            cv2.circle(img, (i[0], i[1]), i[2], (0, 255, 0), 2)
            cv2.circle(img, (i[0], i[1]), 2, (0, 0, 255), 3)

    display_image("Hough Circle Transform", img)

In [7]:
#################################
# 5. Ekstraksi Warna Dominan pada Gambar
#################################
def extract_dominant_color(image_path, k=3):
    img = cv2.imread(image_path)
    data = img.reshape((-1, 3))

    kmeans = KMeans(n_clusters=k)
    kmeans.fit(data)
    colors = kmeans.cluster_centers_.astype(int)

    palette = np.zeros((50, 300, 3), dtype=np.uint8)
    steps = 300 // k
    for i, color in enumerate(colors):
        palette[:, i * steps:(i + 1) * steps, :] = color

    display_image("Dominant Colors", palette)

In [8]:
#################################
# 6. Deteksi Kontur pada Gambar
#################################
def detect_contours(image_path):
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150)

    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cv2.drawContours(img, contours, -1, (0, 255, 0), 2)

    display_image("Contours Detection", img)